# Joint definition of cell types from multiple scRNA-seq datasets (python version)

This notebook demonstrates the usage of the pyliger package.

In [ ]:
from pyliger import *

# Stage I: Preprocessing and Normalization (3 - 5 seconds)

0. Example of reading in 10X CellRanger output using read10X function. 

In [ ]:
sample_dirs = ['10x_ctrl_outs', '10x_stim_outs']
sample_names = ['ctrl', 'stim']

matrix_list = read10X(sample_dirs, sample_names, merge = False)

1.  

2. Initialize a pyliger object using createLiger function.

In [ ]:
ifnb_liger = createLiger(matrix_list)

3. 

In [ ]:
ifnb_liger = normalize(ifnb_liger)
ifnb_liger = selectGenes(ifnb_liger)
ifnb_liger = scaleNotCenter(ifnb_liger)